In [1]:
import spacy

In [2]:
spacy_trans = spacy.load('en')

In [3]:
def read_lines(file_path):
    files = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            files.append(line.strip())
    return files

In [4]:
test_source_path = '/home/ml/cadencao/XSum/fairseq_files/test.source'
test_preds_path = 'preds/xsum_preds.hypo'

In [5]:
xsum_source = read_lines(test_source_path)
xsum_preds = read_lines(test_preds_path)
assert len(xsum_source) == len(xsum_preds)
print(len(xsum_preds))

11301


In [6]:
test_source = spacy_trans(xsum_source[2427])
test_target = spacy_trans(xsum_preds[2427])

In [7]:
print(test_source.ents)

(the Republic of Ireland, Bon Secours, Tuam, County Galway, three year olds, up to 800, Amnesty International, Northern Ireland, Archbishop Martin, Tuam, Church, 2014, first, recent days, Church, Ireland, Church, Irish, Katherine Zappone, 474, between 1940 and 1965, Irish, 2011, Ms Zappone, Irish, decades, Tuam, the end of March, Archbishop Martin, the Catholic Church)


In [8]:
print(test_target.ents)

(Archbishop, Dublin, Justin Martin, the Catholic Church, Ireland)


In [9]:
test_source

These are external links and will open in a new window. The call came after "significant human remains" were found at the site of a former home in the Republic of Ireland. The home was run by the Bon Secours order of nuns in Tuam, County Galway. The bodies ranged from premature babies to three year olds. The discovery was made as part of an investigation into claims by a local historian that up to 800 babies and young children died at the home and were buried in unmarked graves. Amnesty International has said that archaeological surveys should be carried out at all former mother-and-baby homes in Northern Ireland. Archbishop Martin said many in the church and society were "ashamed" of what had emerged at the home in Tuam. He added that "families are owed an apology" and that the Church had repeated an apology it made in 2014 when the claims "first came to light". "It makes me feel awful. I feel incredibly sad in recent days. We as a Church do not want to repeat the awful mistakes of th

In [10]:
test_target

The Archbishop of Dublin, the Most Reverend Justin Martin, has said the Catholic Church in Ireland should apologise to the families of unwed mothers who died at mother-and-baby homes.

In [11]:
"Justin Martin".split()[-1]

'Martin'

In [12]:
test_target_text = test_target.text

test_target_ents = [ent.text for ent in test_target.ents]
test_source_ents = [ent.text for ent in test_source.ents]

for ent in test_target_ents:
    if ent not in test_source_ents:
        if ent.split()[-1] in test_source.text:
            test_target_text = test_target_text.replace("{}".format(ent), ent.split()[-1])
        else:
            test_target_text = test_target_text.replace("{}".format(ent), '')
test_target_text = test_target_text.strip()
test_target_text = test_target_text.replace("  ", " ")

In [13]:
test_target_text

'The Archbishop of , the Most Reverend Martin, has said the Catholic Church in Ireland should apologise to the families of unwed mothers who died at mother-and-baby homes.'

#### Baseline

In [14]:
from tqdm import tqdm

In [15]:
processed_text = []
for s, p in tqdm(zip(xsum_source, xsum_preds)):
    s, p = spacy_trans(s), spacy_trans(p)
    s_ents, p_ents = [ent.text for ent in s.ents], [ent.text for ent in p.ents]

    p_text = p.text
    for e in p_ents:
        if e not in s_ents:
            if e.split()[-1] in s.text:
                p_text = p_text.replace("{}".format(e), e.split()[-1])
            else:
                p_text = p_text.replace("{}".format(e), '')

    p_text = p_text.strip().replace("  ", " ")
    processed_text.append(p_text)

11301it [09:24, 20.02it/s]


In [16]:
len(processed_text)

11301

In [17]:
processed_text[12]

'player Tickle is recovering in hospital after being assaulted in a nightclub in Manchester.'

In [18]:
xsum_preds[12]

'Super League player Paul Tickle is recovering in hospital after being assaulted in a nightclub in Greater Manchester.'

In [19]:
xsum_source[12]

'The Widnes Viking star was taken to hospital with head injuries following an attack at about 04:05 GMT on Sunday. A 32-year-old man has been arrested on suspicion of assault and bailed until 29 November. Greater Manchester Police said they were investigating the incident at Queen\'s nightclub on Wigan Road in Ashton-in-Makerfield. Widnes Viking said they had been "advised that [Tickle] will make a complete recovery", adding they would make no further comments on the matter. The 32-year-old former England international joined the club on a three-year contract in 2014 from Hull FC. His career also involved stints with Wigan Warriors and Halifax.'

In [20]:
with open('preds/processed_preds.txt', 'w') as file:
    for l in processed_text:
        file.write(l + '\n')